In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
%gui qt

In [5]:
from spiketag.base import ProbeFactory

In [6]:
nCh = 3
fs  = 20000.
tritrode = ProbeFactory.genLinearProbe(fs=fs, n_ch=nCh)
tritrode.reorder_by_chip = False

In [7]:
spk = np.fromfile('./cell_0109.bin.spk', dtype=np.int32)
spk.reshape(-1, 2)

array([[   4131,       1],
       [   4458,       1],
       [   6466,       1],
       ...,
       [9278006,       1],
       [9278135,       1],
       [9279424,       1]], dtype=int32)

In [8]:
from spiketag.base import *

In [9]:
from spiketag.view import *

In [19]:
mua = MUA('./cell_0109.bin', tritrode, binary_radix=0)

2018-05-01 17:58:34,247 - spiketag - INFO - #############  load data  ###################
2018-05-01 17:58:34,249 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2018-05-01 17:58:34,251 - spiketag - INFO - 9280040 * 3 points (111360480 bytes) 
2018-05-01 17:58:34,254 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2018-05-01 17:58:34,259 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2018-05-01 17:58:34,260 - spiketag - INFO - #############################################
2018-05-01 17:58:34,380 - spiketag - INFO - processing folder: ./
2018-05-01 17:58:34,383 - spiketag - INFO - raw data have 9309 spks
2018-05-01 17:58:34,385 - spiketag - INFO - ----------------success------------------
2018-05-01 17:58:34,387 - spiketag - INFO -  


In [20]:
mua.pivotal_pos[0]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [21]:
spks = np.fromfile('./cell_0109.spk.bin', dtype=np.int32)

In [23]:
spks[::2]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [24]:
np.allclose(mua.pivotal_pos[0], spks[::2])

True

## Unit test for individual views

In [25]:
wview = wave_view(mua.data, spks=spks)
wview.show()

In [26]:
spk = mua.tospk()

2018-05-01 17:59:20,703 - spiketag - INFO - mua.tospk()
2018-05-01 17:59:21,243 - spiketag - INFO - ----------------success------------------
2018-05-01 17:59:21,244 - spiketag - INFO -  


In [27]:
fet = spk.tofet()

2018-05-01 17:59:22,591 - spiketag - INFO - spk._tofet(groupNo=1, method=weighted-pca, ncomp=6, whiten=False)
2018-05-01 17:59:22,593 - spiketag - INFO - ----------------success------------------
2018-05-01 17:59:22,594 - spiketag - INFO -  


In [28]:
clu = fet.toclu()

2018-05-01 17:59:24,768 - spiketag - INFO - clustering finished, used 0.621437072754 sec


In [41]:
spkview = spike_view()

In [42]:
spkview.set_data(spk.spk[1], clu[1])

In [43]:
spkview.show()

In [44]:
fetview = scatter_3d_view()

In [45]:
fetview.set_data(fet.fet[1], clu[1])

In [46]:
fetview.show()

In [72]:
ampview = amplitude_view(fs=fs, scale=1)

In [73]:
ampview.set_data(spk.spk[1], clu[1], mua.pivotal_pos[0])

In [74]:
ampview.show()

In [51]:
treeview = ctree_view()

In [52]:
treeview.set_data(clu[1])

In [53]:
treeview.show()

In [54]:
corview = correlogram_view(fs=fs)

In [55]:
corview.set_data(clu[1], mua.pivotal_pos[0])

In [56]:
corview.show()

In [57]:
traceview = trace_view(fs=fs, spklen=19)

In [58]:
traceview.set_data(mua.data, clu[1], mua.pivotal_pos[0])

In [59]:
traceview.show()

## Unit test for QT5 Wrapper

In [60]:
import sys
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *

class sorter(QWidget):

    def __init__(self):
        super(sorter, self).__init__()
        self.initUI()

    def initUI(self):

        hbox = QHBoxLayout(self)
        self.splitter1 = QSplitter(Qt.Horizontal)
#         textedit = QTextEdit()
#         self.splitter1.addWidget(self.topleft)
#         self.splitter1.addWidget(textedit)
#         self.splitter1.setSizes([100,200])
        self.splitter2 = QSplitter(Qt.Horizontal)
        self.splitter_fet = QSplitter(Qt.Vertical)

        self.splitter3 = QSplitter(Qt.Vertical)
        self.splitter3.addWidget(self.splitter1)
        self.splitter3.addWidget(self.splitter2)
#         self.splitter2.addWidget(self.bottom)

        hbox.addWidget(self.splitter3)

        self.setLayout(hbox)
        QApplication.setStyle(QStyleFactory.create('Cleanlooks'))

        self.setGeometry(300, 300, 300, 200)
        self.setWindowTitle('spiketag')
#         self.show()

    def set_data(self, mua, spk, fet, clu):
        ### init view and set_data
        self.spkview = spike_view()
        self.spkview.set_data(spk, clu)
        self.fetview0 = scatter_3d_view()
        self.fetview0.set_data(fet[:,[0,1,2]].copy(), clu)
        self.fetview1 = scatter_3d_view()
        self.fetview1.set_data(fet[:,[1,3,4]].copy(), clu)
        self.ampview = amplitude_view(fs=fs, scale=1)
        self.ampview.set_data(spk, clu, mua.pivotal_pos[0])
        self.treeview = ctree_view()
        self.treeview.set_data(clu)
        self.corview = correlogram_view(fs=fs)
        self.corview.set_data(clu, mua.pivotal_pos[0])
        self.traceview = trace_view(fs=mua.fs, spklen=spk.shape[1])
        self.traceview.set_data(mua.data, clu, mua.pivotal_pos[0])
#         self.traceview.locate_buffer = 2000
        
        ### put views into splitter
        self.splitter1.addWidget(self.traceview.native)
        self.splitter1.addWidget(self.splitter_fet)
        self.splitter_fet.addWidget(self.fetview0.native)
        self.splitter_fet.addWidget(self.fetview1.native)
        self.splitter1.addWidget(self.spkview.native)
        
        self.splitter2.addWidget(self.corview.native)
        self.splitter2.addWidget(self.treeview.native)
        self.splitter2.addWidget(self.ampview.native)


In [61]:
%gui qt

In [62]:
ex = sorter()

In [63]:
spk = mua.tospk()
fet = spk.tofet(method='pca', ncomp=9, whiten=False)
clu = fet.toclu(method='hdbscan')

2018-05-01 18:01:12,518 - spiketag - INFO - mua.tospk()
2018-05-01 18:01:12,526 - spiketag - INFO - ----------------success------------------
2018-05-01 18:01:12,530 - spiketag - INFO -  
2018-05-01 18:01:12,563 - spiketag - INFO - spk._tofet(groupNo=1, method=pca, ncomp=9, whiten=False)
2018-05-01 18:01:12,565 - spiketag - INFO - ----------------success------------------
2018-05-01 18:01:12,566 - spiketag - INFO -  
2018-05-01 18:01:13,470 - spiketag - INFO - clustering finished, used 0.902106046677 sec


In [64]:
ex.set_data(mua, spk.spk[1], fet.fet[1], clu[1])

In [66]:
ex.show()

In [70]:
ex.traceview.locate_buffer = 300

In [97]:
ex.fetview0.set_data(fet.fet[1][:,[1,3,4]], clu[1])
ex.update()

ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4774b690>> repeat 8192
ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4a0aa110>> repeat 8192


In [20]:
gclu = clu[1]

In [21]:
gclu.selectlist

array([ 848, 2681, 2795, 2899, 2951, 3162, 3521, 4018, 4031, 4074, 4169,
       4224, 4237, 4486, 4489, 4648, 5064, 5078, 5823, 5966, 5967, 6047,
       6049, 6070, 6610, 7207, 7319, 7919, 8057, 8167, 8378, 8602, 8679,
       9240])

2018-03-11 15:08:33,804 - spiketag - DEBUG - ctree view expand clustier 9311 here
2018-03-11 15:08:51,809 - spiketag - DEBUG - ctree view expand clustier 9313 here


In [21]:
gclu._registered_func_name('select')

['spiketag.view.spike_view.on_select_id121320795744',
 'spiketag.view.scatter_3d_view.on_select_id121444857928',
 'spiketag.view.scatter_3d_view.on_select_id121449105600',
 'spiketag.view.amplitude_view.on_select_id121452650320',
 'spiketag.view.trace_view.on_select_id121529220464']

2018-03-11 14:14:28,666 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:29,401 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:31,183 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:32,756 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,080 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,932 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:35,486 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,014 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,999 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:37,535 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:39,278 - spiketag - DEBUG - ctree view expand clustier 9312 here
2018-03-11 14:14:43,798 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-